# Deepfake Detection Platform - Video Models Test

This notebook tests all video detection models and the Temporal Sentinel Singularity Mode.

In [2]:
# Setup imports
import os
import sys
import logging
import numpy as np
import matplotlib.pyplot as plt
import torch
from IPython.display import Video, display

# Add the project root to the path
project_root = os.path.abspath(os.path.join(os.path.dirname("__file__"), '..'))
sys.path.insert(0, project_root)

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

In [3]:
# Check CUDA availability
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU count: {torch.cuda.device_count()}")
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")

CUDA available: False


## 1. Load Individual Video Detector Models

We'll load all the video detector models: GenConvit, TimeSformer, SlowFast, Video Swin, and X3D.

In [ ]:
# Import video detector models
from detectors.video_detector.genconvit import GenConvitVideoDetector
from detectors.video_detector.timesformer import TimeSformerVideoDetector
from detectors.video_detector.slowfast import SlowFastVideoDetector
from detectors.video_detector.video_swin import VideoSwinDetector
from detectors.video_detector.x3d import X3DVideoDetector

# Define the device to use
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# Initialize the video detectors
try:
    print("Loading GenConvit model...")
    genconvit_detector = GenConvitVideoDetector(model_name="facebook/genconvit-base", 
                                     confidence_threshold=0.5,
                                     device=device)
    
    print("Loading TimeSformer model...")
    timesformer_detector = TimeSformerVideoDetector(model_name="facebook/timesformer-base-224", 
                                     confidence_threshold=0.5,
                                     device=device)
    
    print("Loading SlowFast model...")
    slowfast_detector = SlowFastVideoDetector(model_name="facebook/slowfast-r50", 
                                   confidence_threshold=0.5,
                                   device=device)
    
    print("Loading Video Swin model...")
    video_swin_detector = VideoSwinDetector(model_name="microsoft/swin-tiny-video", 
                                   confidence_threshold=0.5,
                                   device=device)
    
    print("Loading X3D model...")
    x3d_detector = X3DVideoDetector(model_name="facebook/x3d-m", 
                          confidence_threshold=0.5,
                          device=device)
    
    print("All models loaded successfully!")
    
except Exception as e:
    print(f"Error loading models: {str(e)}")

ModuleNotFoundError: No module named 'av'

: 

## 2. Test Individual Models on Sample Videos

We'll test each model on sample real and fake videos.

In [ ]:
# Define test videos
real_video_path = os.path.join(project_root, 'tests', 'test_data', 'Real_Video', 'real_video_sample.mp4')
fake_video_path = os.path.join(project_root, 'tests', 'test_data', 'Fake_Video', 'fake_video_sample.mp4')

# Check if videos exist
if not os.path.exists(real_video_path):
    raise FileNotFoundError(f"Real video not found: {real_video_path}")
if not os.path.exists(fake_video_path):
    raise FileNotFoundError(f"Fake video not found: {fake_video_path}")

# Display the test videos
print("Real Video:")
display(Video(real_video_path, width=320))
print("\nFake Video:")
display(Video(fake_video_path, width=320))

In [ ]:
# Helper function to test models on videos
def test_video_model(detector, video_path, video_type):
    print(f"Testing {detector.__class__.__name__} on {video_type} video...")
    try:
        result = detector.detect(video_path)
        is_deepfake = result['is_deepfake']
        confidence = result['confidence']
        correct = (is_deepfake and video_type == 'fake') or (not is_deepfake and video_type == 'real')
        
        print(f"  Prediction: {'Deepfake' if is_deepfake else 'Real'}")
        print(f"  Confidence: {confidence:.4f}")
        print(f"  Correct: {'✓' if correct else '✗'}")
        
        # Display temporal heatmap if available
        if 'temporal_heatmap' in result:
            plt.figure(figsize=(10, 4))
            plt.plot(result['temporal_heatmap'])
            plt.title(f"Temporal Heatmap for {video_type} Video")
            plt.xlabel("Frame Index")
            plt.ylabel("Deepfake Probability")
            plt.axhline(y=0.5, color='r', linestyle='--')
            plt.grid(True)
            plt.tight_layout()
            plt.show()
        
        return result, correct
    except Exception as e:
        print(f"  Error: {str(e)}")
        return None, False

## Part 2: Test all individual models

In [ ]:
# Part 2: Test all individual models

# Test GenConvit model
genconvit_real_result, genconvit_real_correct = test_video_model(genconvit_detector, real_video_path, 'real')
genconvit_fake_result, genconvit_fake_correct = test_video_model(genconvit_detector, fake_video_path, 'fake')

# Test TimeSformer model
timesformer_real_result, timesformer_real_correct = test_video_model(timesformer_detector, real_video_path, 'real')
timesformer_fake_result, timesformer_fake_correct = test_video_model(timesformer_detector, fake_video_path, 'fake')

# Test SlowFast model
slowfast_real_result, slowfast_real_correct = test_video_model(slowfast_detector, real_video_path, 'real')
slowfast_fake_result, slowfast_fake_correct = test_video_model(slowfast_detector, fake_video_path, 'fake')

# Test Video Swin model
video_swin_real_result, video_swin_real_correct = test_video_model(video_swin_detector, real_video_path, 'real')
video_swin_fake_result, video_swin_fake_correct = test_video_model(video_swin_detector, fake_video_path, 'fake')

# Test X3D model
x3d_real_result, x3d_real_correct = test_video_model(x3d_detector, real_video_path, 'real')
x3d_fake_result, x3d_fake_correct = test_video_model(x3d_detector, fake_video_path, 'fake')


## Part 3: Test Video Ensemble Detector

In [ ]:
# Part 3: Test Video Ensemble Detector

# Import the video ensemble detector
from detectors.video_detector.ensemble import VideoEnsembleDetector

# Create the ensemble detector with all individual detectors
detectors = [genconvit_detector, timesformer_detector, slowfast_detector, video_swin_detector, x3d_detector]
video_ensemble = VideoEnsembleDetector(
    detectors=detectors,
    weights=None,  # Use equal weights initially
    threshold=0.5,
    enable_singularity=False  # First test without Singularity Mode
)

print(f"Created Video Ensemble Detector with {len(detectors)} models")

# Test ensemble on real video
print("Testing Video Ensemble on real video...")
ensemble_real_result = video_ensemble.predict(real_video_path)
is_deepfake = ensemble_real_result['is_deepfake']
confidence = ensemble_real_result['confidence']
print(f"  Prediction: {'Deepfake' if is_deepfake else 'Real'}")
print(f"  Confidence: {confidence:.4f}")
print(f"  Correct: {'✓' if not is_deepfake else '✗'}")

# Show individual model contributions if available
if 'individual_results' in ensemble_real_result:
    print("\nIndividual model contributions:")
    for result in ensemble_real_result['individual_results']:
        model_name = result['model']
        confidence = result['confidence']
        weight = result['weight']
        print(f"  {model_name}: Confidence = {confidence:.4f}, Weight = {weight:.2f}")

# Test ensemble on fake video
print("Testing Video Ensemble on fake video...")
ensemble_fake_result = video_ensemble.predict(fake_video_path)
is_deepfake = ensemble_fake_result['is_deepfake']
confidence = ensemble_fake_result['confidence']
print(f"  Prediction: {'Deepfake' if is_deepfake else 'Real'}")
print(f"  Confidence: {confidence:.4f}")
print(f"  Correct: {'✓' if is_deepfake else '✗'}")

# Show individual model contributions if available
if 'individual_results' in ensemble_fake_result:
    print("\nIndividual model contributions:")
    for result in ensemble_fake_result['individual_results']:
        model_name = result['model']
        confidence = result['confidence']
        weight = result['weight']
        print(f"  {model_name}: Confidence = {confidence:.4f}, Weight = {weight:.2f}")


## Part 4: Test Temporal Sentinel Singularity Mode

In [ ]:
# Part 4: Test Temporal Sentinel Singularity Mode

# Enable Singularity Mode in the ensemble detector
video_ensemble_with_singularity = VideoEnsembleDetector(
    detectors=detectors,
    weights=None,
    threshold=0.5,
    enable_singularity=True  # Enable Singularity Mode
)

print("Created Video Ensemble Detector with Temporal Sentinel Singularity Mode enabled")

# Test Singularity Mode on real video
print("Testing Temporal Sentinel on real video...")
try:
    singularity_real_result = video_ensemble_with_singularity.predict(real_video_path)
    is_deepfake = singularity_real_result['is_deepfake']
    confidence = singularity_real_result['confidence']
    print(f"  Prediction: {'Deepfake' if is_deepfake else 'Real'}")
    print(f"  Confidence: {confidence:.4f}")
    print(f"  Correct: {'✓' if not is_deepfake else '✗'}")
    
    # Show Singularity Mode information if available
    if 'singularity_mode' in singularity_real_result:
        sm_info = singularity_real_result['singularity_mode']
        print("\nSingularity Mode information:")
        for key, value in sm_info.items():
            if key != 'adaptive_weights':  # Weights would be too verbose
                print(f"  {key}: {value}")
        
        if 'adaptive_weights' in sm_info:
            print("  Adaptive weights:")
            for model, weight in sm_info['adaptive_weights'].items():
                print(f"    {model}: {weight:.4f}")
                
    # Display temporal consistency plot if available
    if 'temporal_consistency' in singularity_real_result:
        plt.figure(figsize=(12, 5))
        plt.plot(singularity_real_result['temporal_consistency'])
        plt.title("Temporal Consistency for Real Video")
        plt.xlabel("Frame Pairs")
        plt.ylabel("Consistency Score")
        plt.axhline(y=0.5, color='r', linestyle='--')
        plt.grid(True)
        plt.tight_layout()
        plt.show()
        
except Exception as e:
    print(f"Error testing Temporal Sentinel: {str(e)}")

# Test Singularity Mode on fake video
print("Testing Temporal Sentinel on fake video...")
try:
    singularity_fake_result = video_ensemble_with_singularity.predict(fake_video_path)
    is_deepfake = singularity_fake_result['is_deepfake']
    confidence = singularity_fake_result['confidence']
    print(f"  Prediction: {'Deepfake' if is_deepfake else 'Real'}")
    print(f"  Confidence: {confidence:.4f}")
    print(f"  Correct: {'✓' if is_deepfake else '✗'}")
    
    # Show Singularity Mode information if available
    if 'singularity_mode' in singularity_fake_result:
        sm_info = singularity_fake_result['singularity_mode']
        print("\nSingularity Mode information:")
        for key, value in sm_info.items():
            if key != 'adaptive_weights':  # Weights would be too verbose
                print(f"  {key}: {value}")
        
        if 'adaptive_weights' in sm_info:
            print("  Adaptive weights:")
            for model, weight in sm_info['adaptive_weights'].items():
                print(f"    {model}: {weight:.4f}")
                
    # Display temporal consistency plot if available
    if 'temporal_consistency' in singularity_fake_result:
        plt.figure(figsize=(12, 5))
        plt.plot(singularity_fake_result['temporal_consistency'])
        plt.title("Temporal Consistency for Fake Video")
        plt.xlabel("Frame Pairs")
        plt.ylabel("Consistency Score")
        plt.axhline(y=0.5, color='r', linestyle='--')
        plt.grid(True)
        plt.tight_layout()
        plt.show()
        
except Exception as e:
    print(f"Error testing Temporal Sentinel: {str(e)}")


## Part 5: Performance Comparison

In [ ]:
# Part 5: Performance Comparison

# Collect all confidence scores
real_video_confidences = {
    'GenConvit': genconvit_real_result['confidence'] if genconvit_real_result else 0,
    'TimeSformer': timesformer_real_result['confidence'] if timesformer_real_result else 0,
    'SlowFast': slowfast_real_result['confidence'] if slowfast_real_result else 0,
    'Video Swin': video_swin_real_result['confidence'] if video_swin_real_result else 0,
    'X3D': x3d_real_result['confidence'] if x3d_real_result else 0,
    'Ensemble': ensemble_real_result['confidence'],
    'Temporal Sentinel': singularity_real_result['confidence']
}

fake_video_confidences = {
    'GenConvit': genconvit_fake_result['confidence'] if genconvit_fake_result else 0,
    'TimeSformer': timesformer_fake_result['confidence'] if timesformer_fake_result else 0,
    'SlowFast': slowfast_fake_result['confidence'] if slowfast_fake_result else 0,
    'Video Swin': video_swin_fake_result['confidence'] if video_swin_fake_result else 0,
    'X3D': x3d_fake_result['confidence'] if x3d_fake_result else 0,
    'Ensemble': ensemble_fake_result['confidence'],
    'Temporal Sentinel': singularity_fake_result['confidence']
}

# Plot comparison
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Real video confidences
models = list(real_video_confidences.keys())
confidences = list(real_video_confidences.values())
axes[0].bar(models, confidences, color=['blue', 'blue', 'blue', 'blue', 'blue', 'green', 'red'])
axes[0].set_title('Real Video Confidence Scores')
axes[0].set_xlabel('Model')
axes[0].set_ylabel('Confidence (lower is better for real)')
axes[0].set_ylim(0, 1)
axes[0].axhline(y=0.5, color='r', linestyle='--')
axes[0].set_xticklabels(models, rotation=45)

# Fake video confidences
models = list(fake_video_confidences.keys())
confidences = list(fake_video_confidences.values())
axes[1].bar(models, confidences, color=['blue', 'blue', 'blue', 'blue', 'blue', 'green', 'red'])
axes[1].set_title('Fake Video Confidence Scores')
axes[1].set_xlabel('Model')
axes[1].set_ylabel('Confidence (higher is better for fake)')
axes[1].set_ylim(0, 1)
axes[1].axhline(y=0.5, color='r', linestyle='--')
axes[1].set_xticklabels(models, rotation=45)

plt.tight_layout()
plt.show()


## Part 6: Conclusion

In [ ]:
# Part 6: Conclusion

# Calculate accuracy and improvement
# For real video (lower confidence is better)
real_video_improvement = max(0, np.mean([v for k, v in real_video_confidences.items() if k not in ['Ensemble', 'Temporal Sentinel']]) - real_video_confidences['Temporal Sentinel'])

# For fake video (higher confidence is better)
fake_video_improvement = max(0, fake_video_confidences['Temporal Sentinel'] - np.mean([v for k, v in fake_video_confidences.items() if k not in ['Ensemble', 'Temporal Sentinel']]))

print("Test Results Summary:")
print(f"Temporal Sentinel performance improvement on real video: {real_video_improvement:.4f} (lower confidence is better)")
print(f"Temporal Sentinel performance improvement on fake video: {fake_video_improvement:.4f} (higher confidence is better)")
print(f"Average improvement: {(real_video_improvement + fake_video_improvement) / 2:.4f}")


## Part 7: Frame-by-Frame Analysis with Singularity Mode

In [ ]:
# Part 7: Frame-by-Frame Analysis with Singularity Mode

# This section shows how the Temporal Sentinel analyzes individual frames
# and tracks inconsistencies across the video

from detectors.video_detector.frame_analyzer import VideoFrameAnalyzer
import cv2

# Create a frame analyzer
frame_analyzer = VideoFrameAnalyzer()

def analyze_video_frames(video_path, is_fake=False):
    """Perform frame-by-frame analysis on a video file"""
    # Extract some frames from the video
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    print(f"Analyzing video: {video_path}")
    print(f"Total frames: {total_frames}, FPS: {fps:.2f}")
    
    # Sample frames at regular intervals
    sample_count = min(8, total_frames)
    frame_indices = [int(i * total_frames / sample_count) for i in range(sample_count)]
    
    frames = []
    for idx in frame_indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame = cap.read()
        if ret:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames.append((idx, frame))
    cap.release()
    
    # Display sampled frames
    fig, axes = plt.subplots(2, 4, figsize=(16, 8))
    axes = axes.flatten()
    
    for i, (idx, frame) in enumerate(frames):
        if i < len(axes):
            axes[i].imshow(frame)
            axes[i].set_title(f"Frame {idx}")
            axes[i].axis('off')
    
    plt.tight_layout()
    plt.suptitle(f"{'Fake' if is_fake else 'Real'} Video - Sample Frames")
    plt.subplots_adjust(top=0.9)
    plt.show()
    
    # Run frame analysis with the Singularity Mode
    results = []
    for idx, frame in frames:
        # Convert frame to format expected by the analyzer
        analysis = frame_analyzer.analyze_frame(frame)
        results.append((idx, analysis))
    
    # Plot deepfake probability for each frame
    frame_indices = [idx for idx, _ in results]
    probabilities = [result['deepfake_probability'] for _, result in results]
    
    plt.figure(figsize=(12, 5))
    plt.plot(frame_indices, probabilities, 'o-', linewidth=2)
    plt.axhline(y=0.5, color='r', linestyle='--')
    plt.title(f"{'Fake' if is_fake else 'Real'} Video - Frame-by-Frame Deepfake Probability")
    plt.xlabel("Frame Index")
    plt.ylabel("Deepfake Probability")
    plt.ylim(0, 1)
    plt.grid(True)
    plt.show()
    
    return frames, results

# Analyze real video
real_frames, real_analysis = analyze_video_frames(real_video_path, is_fake=False)

# Analyze fake video
fake_frames, fake_analysis = analyze_video_frames(fake_video_path, is_fake=True)


## Part 8: Temporal Consistency Analysis with Singularity Mode

In [ ]:
# Part 8: Temporal Consistency Analysis with Singularity Mode

# Analyze temporal consistency between consecutive frames
from app.core.singularity_manager import SingularityManager

# Create a Singularity Manager for advanced analysis
singularity_manager = SingularityManager()

def analyze_temporal_consistency(video_path, is_fake=False):
    """Analyze the temporal consistency of a video using the Singularity Manager"""
    print(f"\nAnalyzing temporal consistency for {'fake' if is_fake else 'real'} video...")
    
    try:
        # Run the full singularity analysis
        analysis_result = singularity_manager.analyze_video(
            video_path, 
            deepfake_type="all",
            detailed_analysis=True
        )
        
        # Extract consistency metrics
        consistency_score = analysis_result.get('temporal_consistency_score', 0)
        frame_consistency = analysis_result.get('frame_consistency', [])
        anomalies = analysis_result.get('detected_anomalies', [])
        
        print(f"Overall temporal consistency score: {consistency_score:.4f}")
        print(f"Number of detected anomalies: {len(anomalies)}")
        
        if frame_consistency and len(frame_consistency) > 1:
            # Plot the frame consistency
            plt.figure(figsize=(12, 5))
            plt.plot(frame_consistency, linewidth=2)
            plt.axhline(y=0.5, color='r', linestyle='--')
            plt.title(f"{'Fake' if is_fake else 'Real'} Video - Frame Consistency")
            plt.xlabel("Frame Pair Index")
            plt.ylabel("Consistency Score")
            plt.ylim(0, 1)
            plt.grid(True)
            plt.show()
        
        # If there are anomalies and frames available, show examples
        if anomalies and 'anomaly_frames' in analysis_result:
            anomaly_frames = analysis_result['anomaly_frames']
            if anomaly_frames:
                num_frames = min(4, len(anomaly_frames))
                fig, axes = plt.subplots(1, num_frames, figsize=(16, 5))
                
                if num_frames == 1:
                    axes = [axes]
                
                for i in range(num_frames):
                    frame = anomaly_frames[i]['frame']
                    frame_idx = anomaly_frames[i]['frame_idx']
                    score = anomaly_frames[i]['anomaly_score']
                    
                    axes[i].imshow(frame)
                    axes[i].set_title(f"Frame {frame_idx}\nAnomaly: {score:.4f}")
                    axes[i].axis('off')
                
                plt.tight_layout()
                plt.suptitle(f"Detected Anomalies in {'Fake' if is_fake else 'Real'} Video")
                plt.subplots_adjust(top=0.85)
                plt.show()
        
        return analysis_result
        
    except Exception as e:
        print(f"Error in temporal consistency analysis: {str(e)}")
        return None

# Analyze real video temporal consistency
real_temporal_analysis = analyze_temporal_consistency(real_video_path, is_fake=False)

# Analyze fake video temporal consistency
fake_temporal_analysis = analyze_temporal_consistency(fake_video_path, is_fake=True)


## Part 9: Final Evaluation and Comparative Analysis

In [ ]:
# Part 9: Final Evaluation and Comparative Analysis

# Compare the effectiveness of standard ensemble vs. Singularity Mode
print("\n== Final Deepfake Detection Evaluation ==\n")

# Function to evaluate accuracy
def evaluate_detection(real_result, fake_result):
    # Real should be classified as not deepfake
    real_correct = not real_result['is_deepfake'] if real_result else False
    # Fake should be classified as deepfake
    fake_correct = fake_result['is_deepfake'] if fake_result else False
    
    accuracy = ((1 if real_correct else 0) + (1 if fake_correct else 0)) / 2
    return accuracy, real_correct, fake_correct

# Standard individual models
model_results = [
    ("GenConvit", evaluate_detection(genconvit_real_result, genconvit_fake_result)),
    ("TimeSformer", evaluate_detection(timesformer_real_result, timesformer_fake_result)),
    ("SlowFast", evaluate_detection(slowfast_real_result, slowfast_fake_result)),
    ("Video Swin", evaluate_detection(video_swin_real_result, video_swin_fake_result)),
    ("X3D", evaluate_detection(x3d_real_result, x3d_fake_result))
]

# Ensemble and Singularity
ensemble_acc = evaluate_detection(ensemble_real_result, ensemble_fake_result)
singularity_acc = evaluate_detection(singularity_real_result, singularity_fake_result)

# Print results
print("Individual Model Accuracy:")
for model_name, (acc, real_correct, fake_correct) in model_results:
    print(f"  {model_name}: {acc:.4f} (Real: {'✓' if real_correct else '✗'}, Fake: {'✓' if fake_correct else '✗'})")

print(f"\nStandard Ensemble Accuracy: {ensemble_acc[0]:.4f} (Real: {'✓' if ensemble_acc[1] else '✗'}, Fake: {'✓' if ensemble_acc[2] else '✗'})")
print(f"Temporal Sentinel Accuracy: {singularity_acc[0]:.4f} (Real: {'✓' if singularity_acc[1] else '✗'}, Fake: {'✓' if singularity_acc[2] else '✗'})")

# Calculate improvement
avg_individual_acc = sum(acc for _, (acc, _, _) in model_results) / len(model_results)
improvement_over_individual = singularity_acc[0] - avg_individual_acc
improvement_over_ensemble = singularity_acc[0] - ensemble_acc[0]

print(f"\nTemporal Sentinel improvement over individual models: {improvement_over_individual:.4f}")
print(f"Temporal Sentinel improvement over standard ensemble: {improvement_over_ensemble:.4f}")

# Plot comparative analysis
models = [name for name, _ in model_results] + ["Ensemble", "Temporal Sentinel"]
accuracies = [acc for _, (acc, _, _) in model_results] + [ensemble_acc[0], singularity_acc[0]]

plt.figure(figsize=(12, 6))
bars = plt.bar(models, accuracies, color=['blue', 'blue', 'blue', 'blue', 'blue', 'green', 'red'])
plt.title('Model Accuracy Comparison')
plt.xlabel('Model')
plt.ylabel('Accuracy')
plt.ylim(0, 1)
plt.axhline(y=0.5, color='r', linestyle='--', alpha=0.5)  # Random guess line
plt.xticks(rotation=45)

# Add a legend
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='blue', label='Individual Models'),
    Patch(facecolor='green', label='Standard Ensemble'),
    Patch(facecolor='red', label='Temporal Sentinel')
]
plt.legend(handles=legend_elements, loc='upper left')

# Add value labels on bars
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 0.01,
            f'{height:.2f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

print("\nThis test notebook demonstrates the effectiveness of combining multiple video-based deepfake detection models")
print("with the Temporal Sentinel Singularity Mode, which provides improved accuracy and detailed temporal analysis.")
